In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets("D:/HLK/mydata/try/MNIST_data/", one_hot=True,reshape=False)
# mnist= input_data.read_data_sets("D:/BaiduNetdiskDownload/DAF/Fashion-MNIST/", one_hot=True,reshape=False)
import matplotlib.pyplot as plt
from tensorflow.contrib.layers.python.layers import batch_norm
import time
from tqdm import tqdm

In [ ]:
is_training=tf.concat(False,1)
def bnconv(x,filters,kernel_size=[3,3],strides=1,is_training=is_training, padding='same'):
    with tf.name_scope('bnconv'):
        x=tf.layers.conv2d(x,filters,kernel_size=kernel_size, strides=(strides, strides), padding=padding,use_bias=True)
        x=batch_norm(x,decay=0.99,updates_collections=None,is_training=is_training,activation_fn=tf.nn.leaky_relu)
        
    return x

In [ ]:
def Autoencoders(image):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('Autoencoders')])>0
    with tf.variable_scope('Autoencoders',reuse=reuse):
        x=bnconv(image,64,[5,5])
        x=bnconv(x,64,[5,5])
        x=bnconv(x,64,[5,5],strides=2)
        x=bnconv(x,64,[5,5])
        x=bnconv(x,64,[5,5])
        x=bnconv(x,128,[5,5],strides=2)
        x=bnconv(x,128,[5,5])
        x=bnconv(x,64,[5,5])
        x=tf.layers.conv2d(x,32,kernel_size=[7,7],strides=(1, 1), padding='valid')
        x=tf.reshape(x,[-1,16,2])
        x=tf.nn.softmax(x,-1)
    return x

In [ ]:
x = tf.placeholder(tf.float32,[None, 28,28,1])
x1 = tf.placeholder(tf.float32,[None, 28,28,1])
y=Autoencoders(x)
y1=Autoencoders(x1)

In [ ]:
t_vars=tf.trainable_variables()
d_clip = [tf.assign(var, tf.clip_by_value(var, -0.01, 0.01)) for var in t_vars]

tf.summary.histogram('y',y)
loss=tf.reduce_mean(tf.abs(tf.abs(y1-y)-0.382))
# loss=0.5*tf.log(loss)+0.5*tf.log(1-loss)
tf.summary.scalar('loss',loss)

train=tf.train.RMSPropOptimizer(0.0001).minimize(loss)
# train=tf.train.AdamOptimizer(0.001).minimize(-loss)

In [ ]:
sess=tf.Session()
merged=tf.summary.merge_all()
writer=tf.summary.FileWriter(r"D:\HLK\jupyter_notebook\GAN\result\A\1",sess.graph)
sess.run(tf.global_variables_initializer())

In [ ]:
for i in tqdm(range(40001)):
    batch_xs1, batch_ys = mnist.train.next_batch(64)
    batch_xs2, batch_ys = mnist.train.next_batch(64)
    sess.run(train,{x:batch_xs1,x1:batch_xs2,is_training:True})
    if i%50==0:
        result=sess.run(merged,feed_dict={x:batch_xs1,x1:batch_xs2})
        writer.add_summary(result,i)

In [ ]:
#tensorboard --logdir=D:\HLK\jupyter_notebook\GAN\result\A\